In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import pickle
import pathlib
import tqdm
import numpy as np
import numpy.random
import pandas as pd
from sklearn import svm
import sklearn.metrics as metr
import sklearn.preprocessing as pp
import sklearn.linear_model as lm
import sklearn.neighbors.nearest_centroid as nc
import sklearn.neighbors as ne
import sklearn.naive_bayes as nb
import sklearn.ensemble as em
import sklearn.discriminant_analysis as da
import sklearn.gaussian_process as gp
from sklearn import tree
from sklearn.metrics import f1_score
from sklearn.base import clone
import sklearn.model_selection as ms
import gestureanalysis.svm_helpers as svmhelper
import sklearn.feature_selection as fs
import sklearn.utils
import matplotlib.pyplot as plt
import seaborn as sns
from gestureanalysis.constants import Constants
from gestureanalysis.fit_utils import fit_classifier
from gestureanalysis.sensor_utils import *
from dataclasses import dataclass
from functools import partial

In [3]:
base_path = "/home/jsimon/Documents/thesis/gesture-analysis/data/"
time_groups_path_corrected_pickl = base_path+"transformed/time_added/all/time-and-groups-corrected-all.pkl"
stats_added_base_path = base_path+"transformed/stats_added/all/"
stats_added_path_pickl = stats_added_base_path+"raw_stats-added-all.pkl"
gyro_calibration_path = base_path+'../scripts/gestureanalysis/gyro_offset.txt'

In [4]:
import os
os.getcwd()

'/home/jsimon/Documents/thesis/gesture-analysis/scripts'

## List of Classifiers to test with

In [5]:
# start with some list, and remove then when the classifier takes too long
classifiers = [("passive agressive", lm.PassiveAggressiveClassifier(max_iter=1000, tol=1e-3, class_weight='balanced', n_jobs=3)),
               ("sgd[hinge]",lm.SGDClassifier(loss="hinge", penalty="l2", max_iter=1000, tol=1e-3, class_weight='balanced', n_jobs=3)),
               ("LDA", da.LinearDiscriminantAnalysis()),
               ("QDA", da.QuadraticDiscriminantAnalysis()),
               ("nn",nc.NearestCentroid()),
               #("knn",ne.KNeighborsClassifier(n_jobs=3)),
               ("gaussian navie bayse",nb.GaussianNB()),
               ("decision tree",tree.DecisionTreeClassifier(class_weight='balanced')), # timed out
               ("random forrest",em.RandomForestClassifier(n_estimators=10, class_weight='balanced', n_jobs=3, verbose=True)),
               ("extra trees",em.ExtraTreesClassifier(n_estimators=10, class_weight='balanced', n_jobs=3, verbose=True)),
               ("ada boost",em.AdaBoostClassifier(n_estimators=100)),
               #("gradient boost",em.GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0, verbose=True)) # timed out, says after 1h running that it still has 400 open
]

## Sensor Selection Algorithm RSE

This is the sensor selection algorithm, described in the master thesis

In [13]:
def select_sensors(alpha, X, y, Xval, yval, features, classifiers, fit_classifier, results):
    if len(features.flatten()) <= 1:
        print("done")
        return
    sensors, featuremap = sensor_array(features)
    if len(sensors[sensors > 0].flatten()) <= 0:
        print("done")
        return
    idx, value = weakest_idx(sensors.flatten())
    print(f"sensor removed: {idx}")
    print(f"sensors inclusive removed: {sensors}")
    features_to_remove = featuremap[idx]
    indices = [i for i, x in enumerate(features) if x not in features_to_remove]
    X = X[:,indices]
    Xval = Xval[:,indices]
    features = features[indices]
    
    if X.shape[1] == 0:
        print("done")
        return
    
    r = fit_classifier(classifiers, X, y, Xval, yval)
    b = best_classifier(r)
    results.append({
        'sensors' : sensors,
        'features' : features,
        'best_classifier' : b
    })
    
    linc = sklearn.linear_model.Lasso(alpha = alpha, max_iter=2000, tol=1e-3)
    linc.fit(X, y)
    rfecv = fs.RFECV(estimator=linc, cv=5, step=0.05, verbose=2)
    rfecv.fit(X, y)
    col_index = rfecv.get_support(indices=True)
    X = X[:,col_index]
    Xval = Xval[:,col_index]
    features = features[col_index]
    select_features(alpha, X, y, Xval, yval, features, classifiers, fit_classifier, results)

this method below is just a legacy wrapper from refactoring, as I first named the method wrong. Remove that as soon as not code uses the wrong name

In [7]:
def select_features(alpha, X, y, Xval, yval, features, classifiers, fit_classifier, results):
    print("TODO: Refactor that code")
    select_sensors(alpha, X, y, Xval, yval, features, classifiers, fit_classifier, results)

### some Helper code to load data, do not have that repeated all over

In [ ]:
# check with index is zero
def zero_idx(arr):
    print(np.where(arr==0)[0])

In [ ]:
# set default params of this notebook for select_sensors_from_bunch
select_sensors_fb = partial(select_sensors_from_bunch, select_sensors=select_sensors, classifiers=classifiers, 
                            fit_classifier=fit_classifier, alpha=0.5)

# Experiment 1, Select Sensors with all data with different baseline datasets

## Data Set rfecv2 / abs 765

In [17]:
db = load_data(stats_added_base_path+'train-data-rfecv2.pk')
res1 = select_sensors_fb(db)

sensor removed: 1
sensors inclusive removed: [[ 17.]
 [  1.]
 [ 15.]
 [ 21.]
 [ 19.]
 [ 22.]
 [ 15.]
 [  7.]
 [  7.]
 [ 15.]
 [ 22.]
 [  9.]
 [ 24.]
 [  4.]
 [  6.]
 [ 15.]
 [ 25.]
 [ 17.]
 [ 36.]
 [130.]
 [ 72.]
 [ 99.]
 [ 32.]
 [120.]
 [ 45.]
 [  0.]]


val score 0.915615906886518 for passive agressive (test: 0.9103981445689988)
val score 0.9130294212738441 for sgd[hinge] (test: 0.9261821930163637)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


val score 0.9183640478499838 for LDA (test: 0.9186702744491689)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py

...

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    1.4s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.2s finished


val score 0.8926608470740381 for random forrest (test: 0.9955031568096895)


[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.3s finished


val score 0.895408988037504 for extra trees (test: 1.0)
val score 0.16505011315874554 for ada boost (test: 0.5645921917278701)
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator

ValueError: Found array with 0 feature(s) (shape=(77610, 0)) while a minimum of 1 is required.

In [20]:
with open( stats_added_base_path+'sensor-selection-abs-765.pkl', "wb" ) as users_pickle_file:
    pickle.dump(results, users_pickle_file)

## Data Set rfecv3 / abs 192

In [14]:
db = load_data(stats_added_base_path+'train-data-rfecv3.pk')
res2 = select_sensors_fb(db)

sensor removed: 6
sensors inclusive removed: [[ 2.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 3.]
 [ 2.]
 [ 1.]
 [ 2.]
 [ 3.]
 [ 2.]
 [ 8.]
 [ 3.]
 [ 2.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 9.]
 [11.]
 [11.]
 [25.]
 [35.]
 [43.]
 [22.]
 [12.]
 [ 0.]
 [ 0.]]


val score 0.47752990623989655 for passive agressive (test: 0.5868187089292617)
val score 0.36517943743937925 for sgd[hinge] (test: 0.5194691405746682)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


val score 0.7607500808276754 for LDA (test: 0.8210282180131426)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py

...

val score 0.48916909149692855 for QDA (test: 0.27500322123437704)
val score 0.21580989330746847 for nn (test: 0.537366318773354)
val score 0.3385063045586809 for gaussian navie bayse (test: 0.19681742043551087)
val score 0.8517620433236339 for decision tree (test: 1.0)


[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    1.9s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.2s finished


val score 0.8986420950533462 for random forrest (test: 0.9950650689344157)


[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.4s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.2s finished


val score 0.8934691238279987 for extra trees (test: 1.0)
val score 0.06013579049466537 for ada boost (test: 0.5192500966370313)

Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting

In [ ]:
with open( stats_added_base_path+'sensor-selection-abs-192.pkl', "wb" ) as users_pickle_file:
    pickle.dump(res2, users_pickle_file)

## Data Set rfe / abs 1910

In [22]:
db = load_data(stats_added_base_path+'train-data-rfe.pkl')
res3 = select_sensors_fb(db)

sensor removed: 1
sensors inclusive removed: [[ 73.]
 [ 23.]
 [ 55.]
 [ 64.]
 [ 57.]
 [ 62.]
 [ 50.]
 [ 38.]
 [ 42.]
 [ 39.]
 [ 66.]
 [ 28.]
 [ 52.]
 [ 36.]
 [ 50.]
 [ 46.]
 [ 49.]
 [ 52.]
 [ 72.]
 [202.]
 [231.]
 [178.]
 [170.]
 [153.]
 [102.]
 [  0.]]


val score 0.3674426123504688 for passive agressive (test: 0.4688313361680196)
val score 0.06256062075654704 for sgd[hinge] (test: 0.31008890606880557)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


val score 0.9327513740704817 for LDA (test: 0.9298028604561268)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py

...

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    1.5s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


val score 0.8913676042677013 for random forrest (test: 0.9936734956835459)


[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.4s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.2s finished


val score 0.8887811186550275 for extra trees (test: 1.0)
val score 0.06999676689298416 for ada boost (test: 0.5287076407679423)
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimato

ValueError: Found array with 0 feature(s) (shape=(77610, 0)) while a minimum of 1 is required.

In [23]:
with open( stats_added_base_path+'sensor-selection-abs-1910.pkl', "wb" ) as users_pickle_file:
    pickle.dump(res3, users_pickle_file)

### Intermediate analysis
just do some analysis, like which indices are zero, how many steps are needed and so on for the runs above (depending on what r is set to)

In [111]:
r = res2
step_num = 0

In [112]:
len(r)

16

In [113]:
zero_idx(r[step_num]['sensors'])

[ 1  2  3 13 14 15 24 25]


In [114]:
len(r[step_num]['sensors'])

26

In [115]:
len(np.where(r[step_num]['sensors']==0)[0])

8

In [116]:
len(r[step_num]['features'])

191

In [117]:
r[step_num]

{'sensors': array([[ 2.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 3.],
        [ 2.],
        [ 1.],
        [ 2.],
        [ 3.],
        [ 2.],
        [ 8.],
        [ 3.],
        [ 2.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 9.],
        [11.],
        [11.],
        [25.],
        [35.],
        [43.],
        [22.],
        [12.],
        [ 0.],
        [ 0.]]),
 'features': array(['4_21_Finger_1_tip_fft_skew', '4_27_Finger_1_tip_ff2',
        '7_1_Finger_3_base_std', '7_2_Finger_3_base_min',
        '8_5_Finger_3_tip_75q', '8_19_Finger_3_tip_fft_range',
        '8_44_Finger_3_tip_cwt_std', '9_32_Finger_4_base_bandwith',
        '9_33_Finger_4_base_cwt_sums_0', '10_17_Finger_4_tip_fft_75q',
        '10_19_Finger_4_tip_fft_range', '10_20_Finger_4_tip_fft_var',
        '10_27_Finger_4_tip_ff2', '10_46_Finger_4_tip_cwt_25q',
        '10_56_Finger_4_tip_peak_min', '11_4_Thumb_tip_median',
        '11_5_Thumb_tip_75q', '12_26_Finger_1_pressure_ff1',
   

## For the paper on sensor selection:

I create some subsets of labels and run the selection

- First subset: swipe up, swipe down, swipe left, swipe right, go away, come here
- Second subset: swipe up, swipe down, swipe left, swipe right, thumbs up, thumbs down
- Third subset: swipe up, swipe down, swipe left, swipe right
- Forth subset: swipe up, swipe left, swipe right

In [58]:
with open( stats_added_base_path+'train-data-rfecv3.pk', "rb" ) as users_pickle_file:
    data = pickle.load(users_pickle_file)
Xrfe = data['X']
y = data['y']
XValrfe = data['Xval']
yval = data['yval']
gestures = data['gestures']
rfe_features = data['headers']

In [18]:
gestures

['(1) One',
 '(2) Two',
 '(3) Three',
 '(4) Four',
 '(5) Five',
 'Calling',
 'Come here',
 'Continue',
 'Cutthroat',
 'Down',
 'Go away',
 'Grasp 1',
 'Grasp 2',
 'Knocking',
 'Money',
 'Never mind',
 'Point',
 'Point to self',
 'Push away',
 'Scissor',
 'Shoot',
 'Shoulder pat',
 'Swipe left',
 'Swipe right',
 'Swipe up',
 'Talking',
 'Thumbs down',
 'Thumbs up',
 'Turn',
 'Up',
 'Walking',
 'Waving',
 'Zoom']

In [15]:
np.where(np.array(gestures) == 'Swipe left')

(array([22]),)

In [16]:
np.where(np.array(gestures) == 'Swipe right')

(array([23]),)

In [17]:
np.where(np.array(gestures) == 'Swipe up')

(array([24]),)

In [12]:
np.where(np.array(gestures) == 'Down')

(array([9]),)

In [19]:
np.where(np.array(gestures) == 'Come here')

(array([6]),)

In [20]:
np.where(np.array(gestures) == 'Go away')

(array([10]),)

In [21]:
np.where(np.array(gestures) == 'Thumbs down')

(array([26]),)

In [22]:
np.where(np.array(gestures) == 'Thumbs up')

(array([27]),)

In [14]:
len(y[y==9])

1126

In [8]:
set1 = [24, 9, 22, 23, 10, 6]
set2 = [24, 9, 22, 23, 27, 26]
set3 = [24, 9, 22, 23]
set4 = [24, 22, 23]

### Experiment one, all swipes w. go away and come here

In [24]:
set1

[24, 9, 22, 23, 10, 6]

In [25]:
mask = np.isin(y, set1, invert=True)

In [26]:
yhat = y.copy()

In [27]:
yhat[mask] = 0

In [29]:
np.unique(yhat)

array([ 0,  6,  9, 10, 22, 23, 24])

In [30]:
mask_val = np.isin(yval, set1, invert=True)

In [31]:
yvalhat = yval.copy()

In [32]:
yvalhat[mask_val] = 0

In [40]:
results=[]
select_features(0.5, Xrfe, yhat, XValrfe, yvalhat, rfe_features.flatten(), classifiers, fit_classifier, results)

sensor removed: 6
sensors inclusive removed: [[ 2.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 3.]
 [ 2.]
 [ 1.]
 [ 2.]
 [ 3.]
 [ 2.]
 [ 8.]
 [ 3.]
 [ 2.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 9.]
 [11.]
 [11.]
 [25.]
 [35.]
 [43.]
 [22.]
 [12.]
 [ 0.]
 [ 0.]]


val score 0.7542838667959909 for passive agressive (test: 0.7972812781858007)
val score 0.7529906239896541 for sgd[hinge] (test: 0.8354722329596701)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


val score 0.8661493695441319 for LDA (test: 0.9307563458317227)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py

...

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.8s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished


val score 0.9523116715163272 for random forrest (test: 0.9943048576214405)


[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished


val score 0.957807953443259 for extra trees (test: 1.0)
val score 0.7984157775622374 for ada boost (test: 0.8996005669372503)

Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator

ValueError: Found array with 0 feature(s) (shape=(77610, 0)) while a minimum of 1 is required.

In [43]:
set1_results1_data_rfecv3_abs = results

In [44]:
with open( stats_added_base_path+'set1-results1-data-rfecv3-abs.pkl', "wb" ) as users_pickle_file:
    pickle.dump(results, users_pickle_file)

In [12]:
def transform_ys(y, yval, comparison_set):
    mask = np.isin(y, comparison_set, invert=True)
    yhat = y.copy()
    yhat[mask] = 0
    mask_val = np.isin(yval, comparison_set, invert=True)
    yvalhat = yval.copy()
    yvalhat[mask_val] = 0
    return yhat, yvalhat

In [54]:
yhat2, yvalhat2 = transform_ys(y, yval, set2)

In [55]:
results=[]
select_features(0.5, Xrfe, yhat2, XValrfe, yvalhat2, rfe_features.flatten(), classifiers, fit_classifier, results)

sensor removed: 6
sensors inclusive removed: [[ 2.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 3.]
 [ 2.]
 [ 1.]
 [ 2.]
 [ 3.]
 [ 2.]
 [ 8.]
 [ 3.]
 [ 2.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 9.]
 [11.]
 [11.]
 [25.]
 [35.]
 [43.]
 [22.]
 [12.]
 [ 0.]
 [ 0.]]


val score 0.844487552537989 for passive agressive (test: 0.8504445303440278)
val score 0.6062075654704171 for sgd[hinge] (test: 0.7673237984795772)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


val score 0.8882961526026512 for LDA (test: 0.9394923334621827)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py

...

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.4s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


val score 0.959262851600388 for random forrest (test: 0.9916634454322897)


[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.2s finished


val score 0.9709020368574199 for extra trees (test: 1.0)
val score 0.7942127384416424 for ada boost (test: 0.8959927844349955)
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features

ValueError: Found array with 0 feature(s) (shape=(77610, 0)) while a minimum of 1 is required.

In [56]:
set2_results1_data_rfecv3_abs = results

In [57]:
with open( stats_added_base_path+'set2-results1-data-rfecv3-abs.pkl', "wb" ) as users_pickle_file:
    pickle.dump(results, users_pickle_file)

In [60]:
yhat3, yvalhat3 = transform_ys(y, yval, set3)

In [61]:
results=[]
select_features(0.5, Xrfe, yhat2, XValrfe, yvalhat2, rfe_features.flatten(), classifiers, fit_classifier, results)

sensor removed: 6
sensors inclusive removed: [[ 2.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 3.]
 [ 2.]
 [ 1.]
 [ 2.]
 [ 3.]
 [ 2.]
 [ 8.]
 [ 3.]
 [ 2.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 9.]
 [11.]
 [11.]
 [25.]
 [35.]
 [43.]
 [22.]
 [12.]
 [ 0.]
 [ 0.]]


val score 0.7652764306498545 for passive agressive (test: 0.8038139415023837)
val score 0.72971225347559 for sgd[hinge] (test: 0.8340162350212602)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


val score 0.8882961526026512 for LDA (test: 0.9394923334621827)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py

...

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.4s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


val score 0.9571613320400906 for random forrest (test: 0.9908774642442985)


[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished


val score 0.9715486582605884 for extra trees (test: 1.0)
val score 0.7942127384416424 for ada boost (test: 0.8959927844349955)
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features

ValueError: Found array with 0 feature(s) (shape=(77610, 0)) while a minimum of 1 is required.

In [62]:
set3_results1_data_rfecv3_abs = results

In [63]:
with open( stats_added_base_path+'set3-results1-data-rfecv3-abs.pkl', "wb" ) as users_pickle_file:
    pickle.dump(results, users_pickle_file)

In [64]:
yhat4, yvalhat4 = transform_ys(y, yval, set4)

In [65]:
results=[]
select_features(0.5, Xrfe, yhat4, XValrfe, yvalhat4, rfe_features.flatten(), classifiers, fit_classifier, results)

sensor removed: 6
sensors inclusive removed: [[ 2.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 3.]
 [ 2.]
 [ 1.]
 [ 2.]
 [ 3.]
 [ 2.]
 [ 8.]
 [ 3.]
 [ 2.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 9.]
 [11.]
 [11.]
 [25.]
 [35.]
 [43.]
 [22.]
 [12.]
 [ 0.]
 [ 0.]]


val score 0.7967992240543162 for passive agressive (test: 0.8016621569385388)
val score 0.6999676689298415 for sgd[hinge] (test: 0.8079628913799768)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


val score 0.9118978338182994 for LDA (test: 0.9546707898466692)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


...

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.5s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


val score 0.9751050759780149 for random forrest (test: 0.9952841128720525)


[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished


val score 0.9809246686065308 for extra trees (test: 1.0)
val score 0.9107662463627546 for ada boost (test: 0.9565906455353691)
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 9 features

ValueError: Found array with 0 feature(s) (shape=(77610, 0)) while a minimum of 1 is required.

In [66]:
set4_results1_data_rfecv3_abs = results

In [67]:
with open( stats_added_base_path+'set3-results1-data-rfecv3-abs.pkl', "wb" ) as users_pickle_file:
    pickle.dump(results, users_pickle_file)

## Take dataset w. 765 features

In [73]:
with open( stats_added_base_path+'train-data-rfecv2.pk', "rb" ) as users_pickle_file:
    data = pickle.load(users_pickle_file)
Xrfe = data['X']
y = data['y']
XValrfe = data['Xval']
yval = data['yval']
gestures = data['gestures']
rfe_features = data['headers']

In [69]:
yhat1, yvalhat1 = transform_ys(y, yval, set1)

In [70]:
results=[]
select_features(0.5, Xrfe, yhat1, XValrfe, yvalhat1, rfe_features.flatten(), classifiers, fit_classifier, results)

sensor removed: 1
sensors inclusive removed: [[ 17.]
 [  1.]
 [ 15.]
 [ 21.]
 [ 19.]
 [ 22.]
 [ 15.]
 [  7.]
 [  7.]
 [ 15.]
 [ 22.]
 [  9.]
 [ 24.]
 [  4.]
 [  6.]
 [ 15.]
 [ 25.]
 [ 17.]
 [ 36.]
 [130.]
 [ 72.]
 [ 99.]
 [ 32.]
 [120.]
 [ 45.]
 [  0.]]


val score 0.9569996766892984 for passive agressive (test: 0.9585878108491174)
val score 0.9668606530876172 for sgd[hinge] (test: 0.9719752609199845)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


val score 0.9613643711606854 for LDA (test: 0.975003221234377)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py

...

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.5s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


val score 0.9626576139670223 for random forrest (test: 0.9956062363097539)


[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished


val score 0.9639508567733592 for extra trees (test: 1.0)
val score 0.6044293566117038 for ada boost (test: 0.7986599664991625)
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 9 features

ValueError: Found array with 0 feature(s) (shape=(77610, 0)) while a minimum of 1 is required.

In [71]:
set1_results1_data_rfecv2_abs = results

In [72]:
with open( stats_added_base_path+'set1-results1-data-rfecv2-abs.pkl', "wb" ) as users_pickle_file:
    pickle.dump(results, users_pickle_file)

In [ ]:
with open( stats_added_base_path+'train-data-rfecv2.pk', "rb" ) as users_pickle_file:
    data = pickle.load(users_pickle_file)
Xrfe = data['X']
y = data['y']
XValrfe = data['Xval']
yval = data['yval']
gestures = data['gestures']
rfe_features = data['headers']

In [74]:
yhat2, yvalhat2 = transform_ys(y, yval, set2)

In [75]:
results=[]
select_features(0.5, Xrfe, yhat2, XValrfe, yvalhat2, rfe_features.flatten(), classifiers, fit_classifier, results)

sensor removed: 1
sensors inclusive removed: [[ 17.]
 [  1.]
 [ 15.]
 [ 21.]
 [ 19.]
 [ 22.]
 [ 15.]
 [  7.]
 [  7.]
 [ 15.]
 [ 22.]
 [  9.]
 [ 24.]
 [  4.]
 [  6.]
 [ 15.]
 [ 25.]
 [ 17.]
 [ 36.]
 [130.]
 [ 72.]
 [ 99.]
 [ 32.]
 [120.]
 [ 45.]
 [  0.]]


val score 0.957807953443259 for passive agressive (test: 0.9684963277928103)
val score 0.9631425800193987 for sgd[hinge] (test: 0.9722844994201778)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


val score 0.9688005172971226 for LDA (test: 0.9790490916119057)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py

...

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.6s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished


val score 0.9718719689621726 for random forrest (test: 0.9958897049349311)


[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished


val score 0.9734885224700938 for extra trees (test: 1.0)
val score 0.6524409957969609 for ada boost (test: 0.8210926427006828)
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 fe

ValueError: Found array with 0 feature(s) (shape=(77610, 0)) while a minimum of 1 is required.

In [76]:
set2_results1_data_rfecv2_abs = results

In [77]:
with open( stats_added_base_path+'set2-results1-data-rfecv2-abs.pkl', "wb" ) as users_pickle_file:
    pickle.dump(results, users_pickle_file)

In [78]:
with open( stats_added_base_path+'train-data-rfecv2.pk', "rb" ) as users_pickle_file:
    data = pickle.load(users_pickle_file)
Xrfe = data['X']
y = data['y']
XValrfe = data['Xval']
yval = data['yval']
gestures = data['gestures']
rfe_features = data['headers']

In [79]:
yhat3, yvalhat3 = transform_ys(y, yval, set3)

In [80]:
results=[]
select_features(0.5, Xrfe, yhat3, XValrfe, yvalhat3, rfe_features.flatten(), classifiers, fit_classifier, results)

sensor removed: 1
sensors inclusive removed: [[ 17.]
 [  1.]
 [ 15.]
 [ 21.]
 [ 19.]
 [ 22.]
 [ 15.]
 [  7.]
 [  7.]
 [ 15.]
 [ 22.]
 [  9.]
 [ 24.]
 [  4.]
 [  6.]
 [ 15.]
 [ 25.]
 [ 17.]
 [ 36.]
 [130.]
 [ 72.]
 [ 99.]
 [ 32.]
 [120.]
 [ 45.]
 [  0.]]


val score 0.9510184287099903 for passive agressive (test: 0.9557531245973457)
val score 0.9736501778208859 for sgd[hinge] (test: 0.9776188635485118)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


val score 0.973811833171678 for LDA (test: 0.984228836490143)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


...

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.5s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished


val score 0.9713870029097963 for random forrest (test: 0.9960443241850278)


[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished


val score 0.9768832848367281 for extra trees (test: 1.0)
val score 0.8574199806013579 for ada boost (test: 0.929867285143667)
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 9 features.

ValueError: Found array with 0 feature(s) (shape=(77610, 0)) while a minimum of 1 is required.

In [81]:
set3_results1_data_rfecv2_abs = results

In [82]:
with open( stats_added_base_path+'set3-results1-data-rfecv2-abs.pkl', "wb" ) as users_pickle_file:
    pickle.dump(results, users_pickle_file)

In [9]:
with open( stats_added_base_path+'train-data-rfecv2.pk', "rb" ) as users_pickle_file:
    data = pickle.load(users_pickle_file)
Xrfe = data['X']
y = data['y']
XValrfe = data['Xval']
yval = data['yval']
gestures = data['gestures']
rfe_features = data['headers']

In [12]:
yhat4, yvalhat4 = transform_ys(y, yval, set4)

In [13]:
results=[]
select_features(0.5, Xrfe, yhat4, XValrfe, yvalhat4, rfe_features.flatten(), classifiers, fit_classifier, results)

sensor removed: 1
sensors inclusive removed: [[ 17.]
 [  1.]
 [ 15.]
 [ 21.]
 [ 19.]
 [ 22.]
 [ 15.]
 [  7.]
 [  7.]
 [ 15.]
 [ 22.]
 [  9.]
 [ 24.]
 [  4.]
 [  6.]
 [ 15.]
 [ 25.]
 [ 17.]
 [ 36.]
 [130.]
 [ 72.]
 [ 99.]
 [ 32.]
 [120.]
 [ 45.]
 [  0.]]


val score 0.9710636922082121 for passive agressive (test: 0.9710217755443886)
val score 0.9700937601034594 for sgd[hinge] (test: 0.9767813426104883)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


val score 0.9747817652764307 for LDA (test: 0.98590387836619)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


...

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


val score 0.9776915615906887 for random forrest (test: 0.9964437572477773)


[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished


val score 0.9812479793081151 for extra trees (test: 1.0)
val score 0.8889427740058196 for ada boost (test: 0.9480221620925139)

Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 7 feature

ValueError: Found array with 0 feature(s) (shape=(77610, 0)) while a minimum of 1 is required.

In [14]:
set4_results1_data_rfecv2_abs = results

In [15]:
with open( stats_added_base_path+'set3-results1-data-rfecv2-abs.pkl', "wb" ) as users_pickle_file:
    pickle.dump(results, users_pickle_file)

In [ ]:
with open( stats_added_base_path+'train-data-rfe.pkl', "rb" ) as users_pickle_file:
    data = pickle.load(users_pickle_file)
Xrfe = data['X']
y = data['y']
XValrfe = data['Xval']
yval = data['yval']
gestures = data['gestures']
rfe_features = data['headers']

In [9]:
def compute_all_selections(basedataset, basesetshort, sets):
    i = 0
    for s in sets:
        #if pathlib.Path(stats_added_base_path+f'set{i}-results1-data-{basesetshort}-rel.pkl').exists():
        #    print("===== SKIPPING!!!!!! ===== File already exists: ", stats_added_base_path+f'set{i}-results1-data-{basesetshort}-rel.pkl')
        #    i += 1
        #    continue
        with open( stats_added_base_path+basedataset, "rb" ) as users_pickle_file:
            data = pickle.load(users_pickle_file)
        Xrfe = data['X']
        y = data['y']
        XValrfe = data['Xval']
        yval = data['yval']
        gestures = data['gestures']
        rfe_features = data['headers']
        yhat, yvalhat = transform_ys(y, yval, s)
        try:
            results=[]
            select_features(0.5, Xrfe, yhat, XValrfe, yvalhat, rfe_features.flatten(), classifiers, fit_classifier, results)
        except Exception as e:
            print(e)
        with open( stats_added_base_path+f'set{i}-results1-data-{basesetshort}-abs.pkl', "wb" ) as users_pickle_file:
            pickle.dump(results, users_pickle_file)
        i += 1

In [10]:
sets = [set1, set2, set3, set4]

In [13]:
compute_all_selections('train-data-rfecv3.pk', 'rfecv3', sets)

sensor removed: 6
sensors inclusive removed: [[ 2.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 3.]
 [ 2.]
 [ 1.]
 [ 2.]
 [ 3.]
 [ 2.]
 [ 8.]
 [ 3.]
 [ 2.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 9.]
 [11.]
 [11.]
 [25.]
 [35.]
 [43.]
 [22.]
 [12.]
 [ 0.]
 [ 0.]]


val score 0.7714193339799548 for passive agressive (test: 0.8071124855044453)
val score 0.597478176527643 for sgd[hinge] (test: 0.7606365159128978)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


val score 0.8661493695441319 for LDA (test: 0.9307563458317227)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py

...

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.5s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


val score 0.9744584545748465 for random forrest (test: 0.9949104496843191)


[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished


val score 0.9804397025541546 for extra trees (test: 1.0)
val score 0.9107662463627546 for ada boost (test: 0.9565906455353691)

Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 9 feature

Found array with 0 feature(s) (shape=(77610, 0)) while a minimum of 1 is required.



In [14]:
compute_all_selections('train-data-rfecv2.pk', 'rfecv2', sets)

sensor removed: 1
sensors inclusive removed: [[ 17.]
 [  1.]
 [ 15.]
 [ 21.]
 [ 19.]
 [ 22.]
 [ 15.]
 [  7.]
 [  7.]
 [ 15.]
 [ 22.]
 [  9.]
 [ 24.]
 [  4.]
 [  6.]
 [ 15.]
 [ 25.]
 [ 17.]
 [ 36.]
 [130.]
 [ 72.]
 [ 99.]
 [ 32.]
 [120.]
 [ 45.]
 [  0.]]


val score 0.9689621726479146 for passive agressive (test: 0.9726195077953872)
val score 0.9443905593275137 for sgd[hinge] (test: 0.9600953485375596)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


val score 0.9613643711606854 for LDA (test: 0.975003221234377)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py

...

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.


val score 0.9775299062398966 for random forrest (test: 0.9964952969978096)


[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished


val score 0.9827028774652441 for extra trees (test: 1.0)
val score 0.8889427740058196 for ada boost (test: 0.9480221620925139)

Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Fitting estimator with 3 features.
Fitting estimator with 2 features.
Fitting estimator with 7 feature

Found array with 0 feature(s) (shape=(77610, 0)) while a minimum of 1 is required.



In [15]:
compute_all_selections('train-data-rfe.pkl', 'rfe', sets)

sensor removed: 1
sensors inclusive removed: [[ 73.]
 [ 23.]
 [ 55.]
 [ 64.]
 [ 57.]
 [ 62.]
 [ 50.]
 [ 38.]
 [ 42.]
 [ 39.]
 [ 66.]
 [ 28.]
 [ 52.]
 [ 36.]
 [ 50.]
 [ 46.]
 [ 49.]
 [ 52.]
 [ 72.]
 [202.]
 [231.]
 [178.]
 [170.]
 [153.]
 [102.]
 [  0.]]


val score 0.5093760103459425 for passive agressive (test: 0.6348279860842675)
val score 0.24070481732945362 for sgd[hinge] (test: 0.407524803504703)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


val score 0.978176527643065 for LDA (test: 0.9826697590516688)


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/sklearn/discriminant_analysis.py

...

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished


val score 0.9637892014225671 for random forrest (test: 0.9933127174333205)


[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  10 out of  10 | elapsed:    0.0s finished


val score 0.9739734885224701 for extra trees (test: 1.0)
val score 0.9086647268024572 for ada boost (test: 0.9558948589099343)

Fitting estimator with 2 features.
Fitting estimator with 2 features.
Fitting estimator with 2 features.
Fitting estimator with 2 features.
Fitting estimator with 2 features.
sensor removed: 16
sensors inclusive removed: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [2.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]


Found array with 0 feature(s) (shape=(77610, 0)) while a minimum of 1 is required.

